# 02: Line Segment Intersection

*Authors: Jan Erik Swiadek, Prof. Dr. Kevin Buchin*

This notebook serves as supplementary learning material for the course **Geometric Algorithms**.
It showcases implementations of algorithms and data structures presented in the lecture, and it elaborates on some practical considerations concerning their use.
Furthermore, it provides interactive visualisations and animations.

## Table of Contents

1. Setup  
2. Introduction  
3. Algorithms  
    3.1. Brute Force  
    3.2. Plane Sweep  
4. References  

## 1. Setup

First let's import everything we need throughout this notebook, including our module for common geometry operations and data structures as well as our module for visualisation purposes.
The geometry module will probably receive its own notebook later.

TODO: Include generic data structure module (maybe also write 'common geometry primitives and operations' for distinction)

In [ ]:
# Make our modules available for importing
import os
import sys
modules_dir = os.path.abspath("../modules")
if modules_dir not in sys.path:
    sys.path.append(modules_dir)

# xxx module imports
from data_structures import BinaryTree, Comparator, ComparisonResult
from geometry import Point, LineSegment, Intersections
from visualisation import Visualisation, LineSegmentsMode, PointsMode

Additionally, we create a Visualisation object and register a few test instances for it.

In [ ]:
visualisation = Visualisation(400, 400, LineSegment, LineSegmentsMode(draw_vertices = True), PointsMode())

from decimal import Decimal, getcontext
""" getcontext().prec = 20 """

seg1 = LineSegment(Point(Decimal(150), Decimal(25)), Point(Decimal(100), Decimal(300)))
seg2 = LineSegment(Point(Decimal(75), Decimal(150)), Point(Decimal(200), Decimal(250)))
seg3 = LineSegment(Point(Decimal(50), Decimal(250)), Point(Decimal(300), Decimal(125)))
seg4 = LineSegment(Point(Decimal(200), Decimal(100)), Point(Decimal(350), Decimal(225)))
segments = set([seg1, seg2, seg3, seg4])

visualisation.register_example_instance("test", segments)

In [ ]:
visualisation.display()

## 2. Introduction

Set $S = \{s_1,\dotsc,s_n\}$ of (closed) line segments.
Search for all intersection points and the corresponding intersecting segments.

TODO: Explain more, include image.

## 3. Algorithms

### 3.1 Brute Force

Simple brute force algorithm: Just test each line segment pair for intersection.
Running time is obviously $\Theta(n^2)$, which is worst-case optimal.

In [ ]:
from itertools import combinations

def brute_force_lsi(segments: set[LineSegment]) -> Intersections:
    intersections = Intersections()
    segments = list(segments)
    #for i, segment1 in enumerate(segments):
    #    for segment2 in segments[i+1:]:
    for segment1, segment2 in combinations(segments, 2):
        intersection = segment1.intersection(segment2)
        if isinstance(intersection, Point):         # simply ignore the degenerate case (this doesn't work for plane sweep, I think...)
            intersections.add(intersection, (segment1, segment2))
    
    return intersections

In [ ]:
visualisation.register_algorithm("Brute Force", brute_force_lsi)

In [ ]:
visualisation.display()

TODO: Include takeaways

### 3.2 Plane Sweep

Most of the time, not all segment pairs intersect.
Output-sensitive algorithm using the *plane sweep* technique.

TODO: Add explanations. What about degenerate cases?

First, define comparators for data structures. The status structure needs a *dynamic comparator* (TODO: explain; see [Mount, pp. 24--25]).

In [ ]:
from typing import Any, Optional
import sys

class EventQueueComparator(Comparator[Point]):
    def __call__(self, key: Point, item: Any) -> ComparisonResult:
        if not isinstance(item, Point):
            raise TypeError("Event points can only be compared with other points.")
        if key.y > item.y or (key.y == item.y and key.x < item.x):      # TODO: maybe make is_upper method or similar (see [CG, p. 24] for order)
            return ComparisonResult.SMALLER
        elif key == item:
            return ComparisonResult.EQUAL
        else:
            return ComparisonResult.GREATER

class StatusStructureComparator(Comparator[LineSegment]):
    def __init__(self):
        self.event_point: Point = None

    def set_event_point(self, event_point: Point):
        self.event_point = event_point

    def compare_segments(self, key: LineSegment, item: LineSegment) -> ComparisonResult:
        if self.event_point is None:
            raise RuntimeError("Event point not set...")        # Do this differently?
        elif min(key.upper.y, item.upper.y) < self.event_point.y or max(key.lower.y, item.lower.y) > self.event_point.y:
            raise ValueError("Event point has to be in y-ranges of both line segments.")
        elif key.upper.y == key.lower.y:
            if item.upper.y != item.lower.y or (key.upper.x > item.upper.x and key.lower.x > item.lower.x):
                return ComparisonResult.GREATER
            elif key.upper.x < item.upper.x and key.lower.x < item.lower.x:
                return ComparisonResult.SMALLER
            else:
                pass    # TODO: What if one segment is contained in another one? (-> degenerate case)
        elif item.upper.y == item.lower.y:
            return ComparisonResult.SMALLER
        else:
            key_point = key.get_point_at_same_height(self.event_point)
            item_point = item.get_point_at_same_height(self.event_point)
            if abs(key_point.x - item_point.x) < sys.float_info.epsilon:        # Works with Decimal in the example. TODO: Check general case.
                if key.lower.x < item.lower.x:
                    return ComparisonResult.SMALLER
                elif key.lower.x > item.lower.x:
                    return ComparisonResult.GREATER
                else:
                    return ComparisonResult.EQUAL
            elif key_point.x < item_point.x:
                return ComparisonResult.SMALLER
            #elif key_point.x > item_point.x:
            else:
                return ComparisonResult.GREATER

    def __call__(self, key: LineSegment, item: Any) -> ComparisonResult:
        if isinstance(item, LineSegment):
            return self.compare_segments(key, item)
        elif isinstance(item, Point):
            if key.upper.y < item.y or key.lower.y > item.y:
                print(key)
                print(item)
                raise ValueError("Point has to be in y-range of line segment.")
            elif key.upper.y == key.lower.y:
                if key.lower.x < item.x:
                    return ComparisonResult.SMALLER
                elif key.upper.x > item.x:
                    return ComparisonResult.GREATER
                else:
                    return ComparisonResult.EQUAL
            else:
                point = key.get_point_at_same_height(item)
                if abs(point.x - item.x) < sys.float_info.epsilon:      # Works with Decimal in the example. TODO: Check general case.
                    return ComparisonResult.EQUAL
                if point.x < item.x:
                    return ComparisonResult.SMALLER
                #elif point.x > item.x:
                else:
                    return ComparisonResult.GREATER
        else:
            raise TypeError("Status line segments can only be compared with line segments or points.")


Now we can implement the plane sweep algorithm.

In [ ]:
class PlaneSweepLSI:
    def __init__(self, segments: set[LineSegment]):
        self.event_queue: BinaryTree[Point, list[LineSegment]] = BinaryTree(EventQueueComparator())
        for segment in segments:
            def updater(segments: Optional[list[LineSegment]]) -> list[LineSegment]:
                if segments is None:
                    return [segment]
                segments.append(segment)
                return segments
            self.event_queue.update(segment.upper, updater)
            self.event_queue.update(segment.lower, lambda value: value or [])
        self.status_structure_comparator = StatusStructureComparator()
        self.status_structure: BinaryTree[LineSegment, None] = BinaryTree(self.status_structure_comparator)

    def sweep(self) -> Intersections:
        intersections = Intersections()

        while not self.event_queue.is_empty():
            event_point, intersecting_segs = self._handle_event_point()
            if intersecting_segs:
                intersections.add(event_point, intersecting_segs)

        return intersections

    def _handle_event_point(self) -> tuple[Point, set[LineSegment]]:
        result_segments: set[LineSegment] = set()

        """ print(f"Event queue: {event_queue}")
        print(f"Status structure: {status_structure}") """

        p, segments_u = self.event_queue.pop_smallest()

        """ print(f"Event point: {p}")
        print(f"Event queue after pop: {event_queue}")
        print() """

        segments_c, segments_l = [], []

        for seg in self.status_structure.range_between_neighbours(p):
            if seg.lower == p:
                segments_l.append(seg)
            else:
                segments_c.append(seg)

        if len(segments_u) + len(segments_c) + len(segments_l) >= 2:
            result_segments = set((*segments_u, *segments_c, *segments_l))
        for seg in *segments_c, *segments_l:
            self.status_structure.delete(seg)

        self.status_structure_comparator.set_event_point(p)
        for seg in *segments_u, *segments_c:
            self.status_structure.insert(seg, None)

        if not segments_u and not segments_c:
            seg_l = self.status_structure.smaller_neighbour(p)
            seg_r = self.status_structure.greater_neighbour(p)
            if seg_l is not None and seg_r is not None:
                self._find_new_event(seg_l, seg_r, p)
        else:
            seg_dash = self.status_structure.range_between_neighbours(p)[0]          # TODO: maybe we don't need the status structure here...
            seg_l = self.status_structure.smaller_neighbour(seg_dash)
            if seg_l is not None:
                self._find_new_event(seg_l, seg_dash, p)
            seg_dashdash = self.status_structure.range_between_neighbours(p)[-1]     # TODO: maybe we don't need the status structure here...
            seg_r = self.status_structure.greater_neighbour(seg_dashdash)
            if seg_r is not None:
                self._find_new_event(seg_dashdash, seg_r, p)

        return p, result_segments


    def _find_new_event(self, left_segment: LineSegment, right_segment: LineSegment, point: Point):
        intersection = left_segment.intersection(right_segment)
        if isinstance(intersection, Point):         # TODO: What if segments overlap? (-> edge case)
            if intersection.y < point.y or (intersection.y == point.y and intersection.x > point.x):
                self.event_queue.update(intersection, lambda value: value or [])    # TODO: Might already be inserted. Alternatively, use deletion strategy.


In [ ]:
visualisation.register_algorithm("Plane Sweep", lambda x: PlaneSweepLSI(x).sweep())     # TODO: Change this?

In [ ]:
visualisation.display()

TODO: Include Takeaways

# TESTS

In [ ]:
""" seg1 = LineSegment(Point(6, 1), Point(4, 12))
seg2 = LineSegment(Point(3, 6), Point(8, 10))
seg3 = LineSegment(Point(2, 10), Point(12, 5))
seg4 = LineSegment(Point(8, 4), Point(14, 9))
segments = set([seg1, seg2, seg3, seg4]) """

seg1 = LineSegment(Point(Decimal(6), Decimal(1)), Point(Decimal(4), Decimal(12)))
seg2 = LineSegment(Point(Decimal(3), Decimal(6)), Point(Decimal(8), Decimal(10)))
seg3 = LineSegment(Point(Decimal(2), Decimal(10)), Point(Decimal(12), Decimal(5)))
seg4 = LineSegment(Point(Decimal(8), Decimal(4)), Point(Decimal(14), Decimal(9)))
segments = set([seg1, seg2, seg3, seg4])

print(seg1.intersection(seg2))
print()

print(brute_force_lsi(segments))
print()

print(PlaneSweepLSI(segments).sweep())

In [ ]:
point = seg1.intersection(seg2)

print(point)

comp = StatusStructureComparator()
print(comp(seg1, point))
print(comp(seg2, point))

print(seg1.get_point_at_same_height(point))
print(seg2.get_point_at_same_height(point))

## 4. References

[1] Mark de Berg, Otfried Cheong, Marc van Kreveld, and Mark Overmars. *Computational Geometry: Algorithms and Applications*, 3rd Edition, 2008.

[2] David M. Mount. [*CMSC 754: Computational Geometry*](https://www.cs.umd.edu/class/spring2020/cmsc754/Lects/cmsc754-spring2020-lects.pdf), 2020.